# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        # Вычисляем долю значений каждого класса в каждом варианте разбиения
        l_c = l_c / l_s
        r_c = r_c / r_s

        # Вычисляем gini для левого и правго разбиений
        I_SL = np.square(l_c)
        I_SL = (1 - np.sum(I_SL, axis=1)).reshape(-1, 1)

        I_SR = np.square(r_c)
        I_SR = (1 - np.sum(I_SR, axis=1)).reshape(-1, 1)

        # Возвращаем I(Sl)*|Sl|/|S| + I(Sr)*|Sr|/|S|
        return ((l_s*I_SL + r_s*I_SR)/(l_s + r_s)).reshape(-1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        # Вычисляем долю значений каждого класса в каждом варианте разбиения
        l_c = l_c / l_s
        r_c = r_c / r_s

        # Заменяем 0 на 1 для корректного логарифма
        # Вычисляем entropy для левого и правго разбиений
        np.place(l_c, l_c == 0, 1)
        I_SL = l_c * np.log(l_c)
        I_SL = (-np.sum(I_SL, axis=1)).reshape(-1, 1)

        np.place(r_c, r_c == 0, 1)
        I_SR = r_c * np.log(r_c)
        I_SR = (-np.sum(I_SR, axis=1)).reshape(-1, 1)

        # Возвращаем I(Sl)*|Sl|/|S| + I(Sr)*|Sr|/|S|
        return ((l_s*I_SL + r_s*I_SR)/(l_s + r_s)).reshape(-1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        # Вычисляем долю значений каждого класса в каждом варианте разбиения
        l_c = l_c / l_s
        r_c = r_c / r_s

        # Вычисляем misclass для левого и правго разбиений
        I_SL = (1 - np.max(l_c, axis=1)).reshape(-1, 1)

        I_SR = (1 - np.max(r_c, axis=1)).reshape(-1, 1)

        # Возвращаем I(Sl)*|Sl|/|S| + I(Sr)*|Sr|/|S|
        return ((l_s*I_SL + r_s*I_SR)/(l_s + r_s)).reshape(-1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Заносит в sorted_x и sorted_y отсортированные значения фичи и их метки
        # В class_number заносит количество уникальных меток
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Ищет все возможные места разделения значений на две части
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
    
        # Что делает этот блок кода?
        # Все возможные значения разбиения делят sorted_y на отрезки
        # Каждый отрезок состоит из одинаковых меток, в eq_el_count заносится количетсво меток на каждом отрезке
        # В class_increments заносится количетсво меток одного класса на каждом отрезке
        # Также добавляются метки из тех, что не были включены в splitted_sorted_y
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Вычисляет матрицу, где каждая строчка это количество значений в каждом классе
        # Заносит в l_class_count матрицу значений слева, и в r_class_count справа
        # Заносит в l_sizes и r_sizes общее количетсво объектов справа и слева для каждого варианта разбиения
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Для кажого варианта разбиения подсчитывет его качество и выбирает наилучший вариант
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # Заносит в left_el_id количество значений слева для лучшего разбиения
        # Возвращает оценку лучшего разбиения и значение по которому нужно разбивать
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1)
        # Минимальный impurity коэффицент
        min_coef = float('+inf')
        
        # Значение по котрому будет производится деление
        cor_threshold = None
        
        # Фича по которой будем разбивать
        cor_feature_id = None

        # Получеам номера фич для  возможного разбиения
        feature_ids = self.get_feature_ids(self.num_class)

        # Находим лучшее значение разбиения и фичу
        for i in feature_ids:
            coef, threshold = self.__find_threshold(x[:, i], y)
            if coef < min_coef:
                min_coef = coef
                cor_threshold = threshold
                cor_feature_id = i
        
        # Если достигли макисмальной глубины или количество объектов меньше минимального
        # или find_threshold не смог  разбить ни поодной фиче, создаем лист с самаым популярным классом
        if len(y) < self.min_samples_split or depth == self.max_depth or cor_threshold == None :
            class_count = np.bincount(y)
            class_count = np.argsort(class_count)
            self.tree[node_id] = (self.LEAF_TYPE, class_count[-1], 0)
            return

        # Если задан sufficent share, проверям есть ли класс удовлетворяющий ему
        # Если есть, создаем лист с этим классом
        if self.sufficient_share < 1:
            class_count = np.bincount(y)
            class_count_ids = np.argsort(class_count)
            if class_count[class_count_ids[-1]]/len(class_count) >= self.sufficient_share:
                self.tree[node_id] = (self.LEAF_TYPE, class_count_ids[-1], 0)
                return

        # Делим объекты по cor_threshold
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, cor_feature_id, cor_threshold)

        # Если в одну из ветвей вошли все объекты, создаем лист с самым популярным классом
        if x_r.shape[0] == x.shape[0] or x_l.shape[0] == x.shape[0]:
            class_count = np.bincount(y)
            class_count = np.argsort(class_count)
            self.tree[node_id] = (self.LEAF_TYPE, class_count[-1], 0)
            return

        # Создаем узел с cor_feature_id и cor_threshold
        self.tree[node_id] = (self.NON_LEAF_TYPE, cor_feature_id, cor_threshold)
        self.__fit_node(x_r, y_r, 2*node_id + 1, depth + 1)
        self.__fit_node(x_l, y_l, 2*node_id + 2, depth + 1)

    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [11]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

6.16552615166
6.65336704254


## Проверка качества работы

In [7]:
gkf = KFold(n_splits=5, shuffle=True)

In [14]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.931570632743
0.929242537624
0.931362767107
0.930157146421
0.930237392425


In [15]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.892076161969
0.892325600732
0.89190986946
0.890662675646
0.894108842972
